# -Day 1- 搭建开发环境

 转链接https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432170937506ecfb2f6adf8e4757939732f3e32b781c000

# -Day 2- 编写Web App骨架

用aiohttp写一个基本的app.py

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
import asyncio, os, json, time
from datetime import datetime

from aiohttp import web

In [4]:
def index(request):
    return web.Response(body=b'<h1>Awesome</h1>')

@asyncio.coroutine
def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('Get', '/', index)
    srv = yield from loop.create_server(app.make_handler(), '127.0.0.1', 9000)
    logging.info('server started at http://127.0.0.1:9000...')
    return srv

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: loop argument is deprecated
  
/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead
  
INFO:root:server started at http://127.0.0.1:9000...
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:08 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:26 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:04:02 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"


# -Day 3-编写ORM

Object Relational Mapping 对象关系映射

## 创建连接池

In [6]:
import asyncio, os, json, time

@asyncio.coroutine
def create_pool(loop, **kw):
    logging.info('create database connection pool...')
    global __pool
    __pool = yield from aiomysql.create_pool(
        host=kw.get('host', 'localhost'),
        port=kw.get('port', 3306),
        user=kw['user'],
        password=kw['password'],
        db=kw['db'],
        charset=kw.get('charset','utf8'),
        autocommit=kw.get('autocommit', True),
        maxsize=kw.get('maxsize', 10),
        minsize=ke.get('minsize', 1),
        loop=loop
    )

## Select
要执行SELECT语句，我们用select函数执行，需要传入SQL语句和SQL参数

In [7]:
@asyncio.coroutine
def select(sql, args, size=None):
    log(sql, args)
    global __pool
    with (yield from __pool) as conn:
        cur = yield from conn.cursor(aiomysql.DictCursor)
        #将SQL语句的占位符'?'替换为MySql的占位符'%s'
        yield from cur.execute(sql.replace('?', '%s'), args or ())
        if size:
            #通过fetchmany()获取最多指定数量的记录
            rs = yield from cur.fetchmany(size)
        else:
            #否则，通过fetall()获取所有记录
            rs = yield from cur.fetchall()
        yield from cur.close()
        logging.info('rows returned: %s' % len(rs))
        return rs

## Insert, Update, Delete
要执行INSERT、UPDATE、DELETE语句，可以定义一个通用的execute()函数，因为这3种SQL的执行都需要相同的参数，以及返回一个整数表示影响的行数

In [10]:
@asyncio.coroutine
def execute(sql, args):
    log(sql)
    with (yield from __pool) as conn:
        try:
            cur = yield from conn.cursor()
            yield from cur.execute(sql.replace('?', '%s'), args)
            affected = cur.rowcount
            yield from cur.close()
        except BaseException as e:
            raise
        return affected

## ORM

In [13]:
from www.orm import Model, StringField, IntegerField

class User(Model):
    __table__ = 'users'
    
    id = IntegerField(primary_key=True)
    name = StringField()

INFO:root:found model: User (table: users)
INFO:root:  found mapping: id ==> <IntegerField, bigint:None>
INFO:root:  found mapping: name ==> <StringField, varchar(100):None>


In [14]:
# 创建实例:
user = User(id=123, name='Yuan')
# 存入数据库
user.insert()
#查询所有User对象:\
users = User.findAll()

AttributeError: 'Model' object has no attribute 'insert'

In [ ]:
user.